In [ ]:
# SPDX-FileCopyrightText: Copyright (c) 2022 Guillaume Bellegarda. All rights reserved.
# SPDX-License-Identifier: BSD-3-Clause
# 
# Redistribution and use in source and binary forms, with or without
# modification, are permitted provided that the following conditions are met:
#
# 1. Redistributions of source code must retain the above copyright notice, this
# list of conditions and the following disclaimer.
#
# 2. Redistributions in binary form must reproduce the above copyright notice,
# this list of conditions and the following disclaimer in the documentation
# and/or other materials provided with the distribution.
#
# 3. Neither the name of the copyright holder nor the names of its
# contributors may be used to endorse or promote products derived from
# this software without specific prior written permission.
#
# THIS SOFTWARE IS PROVIDED BY THE COPYRIGHT HOLDERS AND CONTRIBUTORS "AS IS"
# AND ANY EXPRESS OR IMPLIED WARRANTIES, INCLUDING, BUT NOT LIMITED TO, THE
# IMPLIED WARRANTIES OF MERCHANTABILITY AND FITNESS FOR A PARTICULAR PURPOSE ARE
# DISCLAIMED. IN NO EVENT SHALL THE COPYRIGHT HOLDER OR CONTRIBUTORS BE LIABLE
# FOR ANY DIRECT, INDIRECT, INCIDENTAL, SPECIAL, EXEMPLARY, OR CONSEQUENTIAL
# DAMAGES (INCLUDING, BUT NOT LIMITED TO, PROCUREMENT OF SUBSTITUTE GOODS OR
# SERVICES; LOSS OF USE, DATA, OR PROFITS; OR BUSINESS INTERRUPTION) HOWEVER
# CAUSED AND ON ANY THEORY OF LIABILITY, WHETHER IN CONTRACT, STRICT LIABILITY,
# OR TORT (INCLUDING NEGLIGENCE OR OTHERWISE) ARISING IN ANY WAY OUT OF THE USE
# OF THIS SOFTWARE, EVEN IF ADVISED OF THE POSSIBILITY OF SUCH DAMAGE.
#
# Copyright (c) 2022 EPFL, Guillaume Bellegarda

"""Motor model for quadrupeds."""

import collections
import numpy as np

NUM_MOTORS = 12

CONTROL_MODES = [ "TORQUE","PD" ]

class QuadrupedMotorModel(object):
  """A simple motor model for a quadruped.

    When in POSITION mode, the torque is calculated according to the difference
    between current and desired joint angle, as well as the joint velocity.
    For more information about PD control, please refer to:
    https://en.wikipedia.org/wiki/PID_controller.

    The model supports a HYBRID mode in which each motor command can be a tuple
    (desired_motor_angle, position_gain, desired_motor_velocity, velocity_gain,
    torque).

  """

  def __init__(self,
               kp=60,
               kd=1,
               torque_limits=None,
               motor_control_mode="PD"):
    self._kp = kp
    self._kd = kd
    self._torque_limits = torque_limits
    if torque_limits is not None:
      if isinstance(torque_limits, (collections.abc.Sequence, np.ndarray)):
        self._torque_limits = np.asarray(torque_limits)
      else:
        self._torque_limits = np.full(NUM_MOTORS, torque_limits)
    self._motor_control_mode = motor_control_mode
    self._strength_ratios = np.full(NUM_MOTORS, 1)

  def convert_to_torque(self,
                        motor_commands,
                        motor_angle,
                        motor_velocity,
                        motor_control_mode=None):
    """Convert the commands (position control or torque control) to torque.

    Args:
      motor_commands: The desired motor angle if the motor is in position
        control mode. The pwm signal if the motor is in torque control mode.
      motor_angle: The motor angle observed at the current time step. It is
        actually the true motor angle observed a few milliseconds ago (pd
        latency).
      motor_velocity: The motor velocity observed at the current time step, it
        is actually the true motor velocity a few milliseconds ago (pd latency).
      motor_control_mode: A MotorControlMode enum.

    Returns:
      actual_torque: The torque that needs to be applied to the motor.
      observed_torque: The torque observed by the sensor.
    """
    if not motor_control_mode:
      motor_control_mode = self._motor_control_mode

    # No processing for motor torques
    # Edit: SHOULD still clip torque values
    if motor_control_mode is "TORQUE":
      assert len(motor_commands) == NUM_MOTORS
      motor_torques = self._strength_ratios * motor_commands
      motor_torques = np.clip(motor_torques, -1.0 * self._torque_limits,
                              self._torque_limits)
      #print('actual motor', motor_torques)
      return motor_torques, motor_torques

    desired_motor_angles = None
    desired_motor_velocities = None
    kp = None
    kd = None
    additional_torques = np.full(NUM_MOTORS, 0)
    if motor_control_mode is "PD":
      assert len(motor_commands) == NUM_MOTORS
      kp = self._kp
      kd = self._kd
      desired_motor_angles = motor_commands
      desired_motor_velocities = np.full(NUM_MOTORS, 0)
    else:
      raise ValueError("Motor model should only be torque or position control.")

    motor_torques = -1 * (kp * (motor_angle - desired_motor_angles)) - kd * (
        motor_velocity - desired_motor_velocities) + additional_torques
    motor_torques = self._strength_ratios * motor_torques
    if self._torque_limits is not None:
      if len(self._torque_limits) != len(motor_torques):
        raise ValueError(
            "Torque limits dimension does not match the number of motors.")
      motor_torques = np.clip(motor_torques, -1.0 * self._torque_limits,
                              self._torque_limits)

    return motor_torques, motor_torques


In [ ]:
# SPDX-FileCopyrightText: Copyright (c) 2022 Guillaume Bellegarda. All rights reserved.
# SPDX-License-Identifier: BSD-3-Clause
# 
# Redistribution and use in source and binary forms, with or without
# modification, are permitted provided that the following conditions are met:
#
# 1. Redistributions of source code must retain the above copyright notice, this
# list of conditions and the following disclaimer.
#
# 2. Redistributions in binary form must reproduce the above copyright notice,
# this list of conditions and the following disclaimer in the documentation
# and/or other materials provided with the distribution.
#
# 3. Neither the name of the copyright holder nor the names of its
# contributors may be used to endorse or promote products derived from
# this software without specific prior written permission.
#
# THIS SOFTWARE IS PROVIDED BY THE COPYRIGHT HOLDERS AND CONTRIBUTORS "AS IS"
# AND ANY EXPRESS OR IMPLIED WARRANTIES, INCLUDING, BUT NOT LIMITED TO, THE
# IMPLIED WARRANTIES OF MERCHANTABILITY AND FITNESS FOR A PARTICULAR PURPOSE ARE
# DISCLAIMED. IN NO EVENT SHALL THE COPYRIGHT HOLDER OR CONTRIBUTORS BE LIABLE
# FOR ANY DIRECT, INDIRECT, INCIDENTAL, SPECIAL, EXEMPLARY, OR CONSEQUENTIAL
# DAMAGES (INCLUDING, BUT NOT LIMITED TO, PROCUREMENT OF SUBSTITUTE GOODS OR
# SERVICES; LOSS OF USE, DATA, OR PROFITS; OR BUSINESS INTERRUPTION) HOWEVER
# CAUSED AND ON ANY THEORY OF LIABILITY, WHETHER IN CONTRACT, STRICT LIABILITY,
# OR TORT (INCLUDING NEGLIGENCE OR OTHERWISE) ARISING IN ANY WAY OUT OF THE USE
# OF THIS SOFTWARE, EVEN IF ADVISED OF THE POSSIBILITY OF SUCH DAMAGE.
#
# Copyright (c) 2022 EPFL, Guillaume Bellegarda

"""
This file implements the functionalities of a quadruped using pybullet.
"""
import os
import numpy as np
import time
import re
import quadruped_motor

class Quadruped(object):
  """The quadruped class to simulate a quadruped robot.
  """
  def __init__(self,
               pybullet_client,
               robot_config=None,
               self_collision_enabled=True,
               accurate_motor_model_enabled=True,
               motor_control_mode="PD",
               on_rack=False,
               render=False):
    """Construct a quadruped and reset it to the initial states.

    Args:
      pybullet_client: The instance of BulletClient to manage different
        simulations.
      robot_config: File with all of the relevant configs for the desired 
        quadruped.
      self_collision_enabled: Whether to enable self collision.
      accurate_motor_model_enabled: Whether to use the accurate DC motor model.
      motor_control_mode: either torque or position control
      on_rack: Whether to place the quadruped on rack. This is only used to debug
        the walking gait. In this mode, the quaruped's base is hanged midair so
        that its walking gait is clearer to visualize.
    """
    self._robot_config = robot_config
    self.num_motors = self._robot_config.NUM_MOTORS
    self.num_legs = self._robot_config.NUM_LEGS 
    self._pybullet_client = pybullet_client
    self._urdf_root = self._robot_config.URDF_ROOT # urdf_root
    self._self_collision_enabled = self_collision_enabled
    self._motor_direction = self._robot_config.JOINT_DIRECTIONS
    self._observed_motor_torques = np.zeros(self.num_motors)
    self._applied_motor_torques = np.zeros(self.num_motors)
    self._accurate_motor_model_enabled = accurate_motor_model_enabled

    # motor control mode for accurate motor model, should only be torque or position at this low level
    if motor_control_mode == "PD":
      self._motor_control_mode = "PD"
    else:
      self._motor_control_mode = "TORQUE"

    self._on_rack = on_rack
    self.render = render
    if self._accurate_motor_model_enabled:
      self._kp = self._robot_config.MOTOR_KP 
      self._kd = self._robot_config.MOTOR_KD
      self._motor_model = quadruped_motor.QuadrupedMotorModel(
                                    motor_control_mode=self._motor_control_mode,
                                    kp=self._kp,
                                    kd=self._kd,
                                    torque_limits=self._robot_config.TORQUE_LIMITS)
    else:
      raise ValueError("Must use accurate motor model")

    self.Reset(reload_urdf=True)


  ######################################################################################
  # Robot states and observation related
  ######################################################################################
  def _GetDefaultInitPosition(self):
    """Get the default initial position of the quadruped's base, to reset simulation

    Returns:
      The initial xyz position of the quadruped's base, either fixed in air, or
        initialized on ground.
    """
    if self._on_rack:
      return self._robot_config.INIT_RACK_POSITION
    else:
      return self._robot_config.INIT_POSITION

  def _GetDefaultInitOrientation(self):
    return self._robot_config.INIT_ORIENTATION

  def GetBasePosition(self):
    """Get the position of the quadruped's base.

    Returns:
      The world xyz position of the quadruped's base.
    """
    position, _ = (self._pybullet_client.getBasePositionAndOrientation(self.quadruped))
    return position

  def GetBaseOrientation(self):
    """Get the orientation of the quadruped's base, represented as quaternion.

    Returns:
      The orientation (quaternion) of the quadruped's base.
    """
    _, orientation = (self._pybullet_client.getBasePositionAndOrientation(self.quadruped))
    _, orientation = self._pybullet_client.multiplyTransforms(
      positionA=[0, 0, 0],
      orientationA=orientation,
      positionB=[0, 0, 0],
      orientationB=self._robot_config.INIT_ORIENTATION_INV)
    return orientation

  def GetBaseOrientationRollPitchYaw(self):
    """Get quadruped's base orientation in euler angle in the world frame.

    Returns:
       (roll, pitch, yaw) of the base in world frame.
    """
    orientation = self.GetBaseOrientation()
    roll_pitch_yaw = self._pybullet_client.getEulerFromQuaternion(orientation)
    return np.asarray(roll_pitch_yaw)

  def GetTrueBaseRollPitchYawRate(self):
    """Get the rate of orientation change of the quadruped's base in euler angle.

    Returns:
      rate of (roll, pitch, yaw) change of the quadruped's base.
    """
    angular_velocity = self.GetBaseAngularVelocity()
    orientation = self.GetBaseOrientation()
    return self.TransformAngularVelocityToLocalFrame(angular_velocity, orientation)

  def TransformAngularVelocityToLocalFrame(self, angular_velocity, orientation):
    """Transform the angular velocity from world frame to robot's frame.

    Args:
    angular_velocity: Angular velocity of the robot in world frame.
    orientation: Orientation of the robot represented as a quaternion.

    Returns:
    angular velocity of based on the given orientation.
    """
    # Treat angular velocity as a position vector, then transform based on the
    # orientation given by dividing (or multiplying with inverse).
    # Get inverse quaternion assuming the vector is at 0,0,0 origin.
    _, orientation_inversed = self._pybullet_client.invertTransform([0, 0, 0], orientation)
    # Transform the angular_velocity at neutral orientation using a neutral
    # translation and reverse of the given orientation.
    relative_velocity, _ = self._pybullet_client.multiplyTransforms([0, 0, 0], 
                            orientation_inversed, 
                            angular_velocity,
                             self._pybullet_client.getQuaternionFromEuler([0, 0, 0]))
    return np.asarray(relative_velocity)

  def GetBaseOrientationMatrix(self):
    """ Get the base orientation matrix (3x3), as numpy array """
    baseOrn = self.GetBaseOrientation()
    return np.asarray(self._pybullet_client.getMatrixFromQuaternion(baseOrn)).reshape((3,3))

  def GetBaseLinearVelocity(self):
    """ Get base linear velocities (dx, dy, dz) """
    linVel,_ = self._pybullet_client.getBaseVelocity(self.quadruped)
    return np.asarray(linVel)

  def GetBaseAngularVelocity(self):
    """ Get base angular velocities (droll, dpitch, dyaw) """
    _, angVel = self._pybullet_client.getBaseVelocity(self.quadruped)
    return np.asarray(angVel)

  def GetMotorAngles(self):
    """Get quadruped motor angles at the current moment.

    Returns:
      Motor angles.
    """
    motor_angles = [
        self._pybullet_client.getJointState(self.quadruped, motor_id)[0]
        for motor_id in self._motor_id_list
    ]
    motor_angles = np.multiply(motor_angles, self._motor_direction)
    return motor_angles

  def GetMotorVelocities(self):
    """Get the velocity of all motors.

    Returns:
      Velocities of all motors.
    """
    motor_velocities = [
        self._pybullet_client.getJointState(self.quadruped, motor_id)[1]
        for motor_id in self._motor_id_list
    ]
    motor_velocities = np.multiply(motor_velocities, self._motor_direction)
    return motor_velocities

  def GetMotorTorques(self):
    """Get the torques the motors are exerting.

    Returns:
      Motor torques of all motors.
    """
    if self._accurate_motor_model_enabled:
      return self._observed_motor_torques
    else:
      motor_torques = [
          self._pybullet_client.getJointState(self.quadruped, motor_id)[3]
          for motor_id in self._motor_id_list
      ]
      motor_torques = np.multiply(motor_torques, self._motor_direction)
    return motor_torques


  def GetContactInfo(self):
    """ Returns 
    (1) num valid contacts (only those between a foot and the ground)
    (2) num invalid contacts (either between a limb and the ground (other than the feet) or any self collisions
    (3) normal force at each foot (0 if in the air) 
    (4) boolean for each foot in contact (1) or not (0)
    """
    numValidContacts = 0
    numInvalidContacts = 0
    feetNormalForces = [0,0,0,0]
    feetInContactBool = [0,0,0,0]
    for c in self._pybullet_client.getContactPoints():
      # if bodyUniqueIdA is same as bodyUniqueIdB, self collision
      if c[1] == c[2]:
        # but only check calves for now
        if ( (c[3] in self._calf_ids ) or (c[4] in self._calf_ids)):
          numInvalidContacts += 1
        continue
      # if thighs in contact with anything, this is a failing condition
      if c[3] in self._thigh_ids or c[4] in self._thigh_ids:
        numInvalidContacts += 1
        continue
      # check link indices, one MUST be a foot index, and the other must be -1 (plane)
      # depending on sim and pybullet version, this can happen in either order
      if ( c[3] == -1 and c[4] not in self._foot_link_ids ) or \
         ( c[4] == -1 and c[3] not in self._foot_link_ids ):
        numInvalidContacts += 1
      else:
        numValidContacts += 1
        try:
          footIndex = self._foot_link_ids.index(c[4])
        except:
          footIndex = self._foot_link_ids.index(c[3])
        feetNormalForces[footIndex] += c[9] # if multiple contact locations
        feetInContactBool[footIndex] = 1
    return numValidContacts, numInvalidContacts, feetNormalForces, feetInContactBool

  ######################################################################################
  # INPUTS: set torques, ApplyAction, etc.
  ######################################################################################
  def _SetMotorTorqueById(self, motor_id, torque):
    self._pybullet_client.setJointMotorControl2(bodyIndex=self.quadruped,
                                                jointIndex=motor_id,
                                                controlMode=self._pybullet_client.TORQUE_CONTROL,
                                                force=torque)

  def _SetDesiredMotorAngleById(self, motor_id, desired_angle): 
    # get max_force 
    max_force = self._robot_config.TORQUE_LIMITS[self._joint_ids.index(motor_id)]
    max_vel = self._robot_config.VELOCITY_LIMITS[self._joint_ids.index(motor_id)]
    self._pybullet_client.setJointMotorControl2(bodyIndex=self.quadruped,
                                                jointIndex=motor_id,
                                                controlMode=self._pybullet_client.POSITION_CONTROL,
                                                targetPosition=desired_angle,
                                                force=max_force,
                                                maxVelocity=max_vel,
                                                )

  def _SetDesiredMotorAngleByName(self, motor_name, desired_angle):
    self._SetDesiredMotorAngleById(self._joint_name_to_id[motor_name], desired_angle)

  def ApplyAction(self, motor_commands):
    """Apply the desired motor torques to the motors of the quadruped.

    Args:
      motor_commands: The desired motor angles or torques, depending on control mode.
    """
    if self._accurate_motor_model_enabled:
      q = self.GetMotorAngles()
      qdot = self.GetMotorVelocities()

      actual_torque, observed_torque = self._motor_model.convert_to_torque(
          motor_commands, q, qdot)
      self._observed_motor_torques = observed_torque

      # Transform into the motor space when applying the torque.
      self._applied_motor_torque = np.multiply(actual_torque, self._motor_direction)

      for motor_id, motor_torque, motor_enabled in zip(self._motor_id_list,
                                                       self._applied_motor_torque,
                                                       self._motor_enabled_list):
        if motor_enabled:
          self._SetMotorTorqueById(motor_id, motor_torque)
        else:
          self._SetMotorTorqueById(motor_id, 0)


  ######################################################################################
  # Jacobian, IK, etc. 
  ######################################################################################
  def ComputeJacobianAndPosition(self, legID):
    """ Get Jacobian and foot position of leg legID. 
    Leg 0: FR; Leg 1: FL; Leg 2: RR ; Leg 3: RL;
    """
    # joint positions of leg legID
    q = self.GetMotorAngles()[legID*3:legID*3+3]

    # rename links
    l1 = self._robot_config.HIP_LINK_LENGTH
    l2 = self._robot_config.THIGH_LINK_LENGTH
    l3 = self._robot_config.CALF_LINK_LENGTH

    sideSign = 1
    if legID == 0 or legID == 2:
      sideSign = -1

    s1 = np.sin(q[0])
    s2 = np.sin(q[1])
    s3 = np.sin(q[2])

    c1 = np.cos(q[0])
    c2 = np.cos(q[1])
    c3 = np.cos(q[2])

    c23 = c2 * c3 - s2 * s3
    s23 = s2 * c3 + c2 * s3

    J = np.zeros((3,3))

    J[1, 0] = -sideSign * l1 * s1 + l2 * c2 * c1 + l3 * c23 * c1
    J[2, 0] =  sideSign * l1 * c1 + l2 * c2 * s1 + l3 * c23 * s1
    J[0, 1] = -l3 * c23 - l2 * c2
    J[1, 1] = -l2 * s2 * s1 - l3 * s23 * s1
    J[2, 1] = l2 * s2 * c1 + l3 * s23 * c1
    J[0, 2] = -l3 * c23
    J[1, 2] = -l3 * s23 *s1
    J[2, 2] = l3 * s23 * c1  

    # foot pos
    pos = np.zeros(3)
    pos[0] = -l3 * s23 - l2 * s2
    pos[1] = l1 * sideSign * c1 + l3 * (s1 * c23) + l2 * c2 * s1
    pos[2] = l1 * sideSign * s1 - l3 * (c1 * c23) - l2 * c1 * c2

    return J, pos

  def ComputeInverseKinematics(self,legID, xyz_coord):
    """ Get joint angles for leg legID with desired xyz position in leg frame. 

    Leg 0: FR; Leg 1: FL; Leg 2: RR ; Leg 3: RL;

    From SpotMicro: 
    https://github.com/OpenQuadruped/spot_mini_mini/blob/spot/spotmicro/Kinematics/LegKinematics.py
    """
    # rename links
    shoulder_length = self._robot_config.HIP_LINK_LENGTH
    elbow_length = self._robot_config.THIGH_LINK_LENGTH
    wrist_length =  self._robot_config.CALF_LINK_LENGTH
    # coords
    x = xyz_coord[0]
    y = xyz_coord[1]
    z = xyz_coord[2]

    # get_domain
    D = (y**2 + (-z)**2 - shoulder_length**2 +
        (-x)**2 - elbow_length**2 - wrist_length**2) / (
                 2 * wrist_length * elbow_length)

    D = np.clip(D, -1.0, 1.0)

    # check Right vs Left leg for hip angle
    sideSign = 1
    if legID == 0 or legID == 2:
      sideSign = -1

    # Right Leg Inverse Kinematics Solver
    wrist_angle = np.arctan2(-np.sqrt(1 - D**2), D)
    sqrt_component = y**2 + (-z)**2 - shoulder_length**2
    if sqrt_component < 0.0:
        sqrt_component = 0.0
    shoulder_angle = -np.arctan2(z, y) - np.arctan2(
        np.sqrt(sqrt_component), sideSign*shoulder_length)
    elbow_angle = np.arctan2(-x, np.sqrt(sqrt_component)) - np.arctan2(
        wrist_length * np.sin(wrist_angle),
        elbow_length + wrist_length * np.cos(wrist_angle))
    joint_angles = np.array([-shoulder_angle, elbow_angle, wrist_angle])
    return joint_angles

  ######################################################################################
  # RESET related
  ######################################################################################
  def Reset(self, reload_urdf=False):
    """Reset the quadruped to its initial states.

    Args:
      reload_urdf: Whether to reload the urdf file. If not, Reset() just place
        the quadruped back to its starting position.
    """
    if reload_urdf:
      self._LoadRobotURDF()
      self._BuildJointNameToIdDict()
      self._BuildUrdfIds()
      self._RemoveDefaultJointDamping()
      self._SetLateralFriction()
      self._BuildMotorIdList()
      self._RecordMassAndInertiaInfoFromURDF()
      self._SetMaxJointVelocities()

      self.ResetPose(add_constraint=True)
      if self._on_rack:
        self._pybullet_client.createConstraint(self.quadruped, -1, -1, -1,
                                               self._pybullet_client.JOINT_FIXED, [0, 0, 0],
                                               [0, 0, 0], [0, 0, 1], childFrameOrientation=self._GetDefaultInitOrientation())
    else:
      self._pybullet_client.resetBasePositionAndOrientation(self.quadruped, 
                                                            self._GetDefaultInitPosition(),
                                                            self._GetDefaultInitOrientation())
      self._pybullet_client.resetBaseVelocity(self.quadruped, [0, 0, 0], [0, 0, 0])
      self.ResetPose(add_constraint=False)

    self._overheat_counter = np.zeros(self.num_motors)
    self._motor_enabled_list = [True] * self.num_motors


  def ResetPose(self, add_constraint):
    """From laikago.py """
    del add_constraint
    for name in self._joint_name_to_id:
      joint_id = self._joint_name_to_id[name]
      self._pybullet_client.setJointMotorControl2(
          bodyIndex=self.quadruped,
          jointIndex=(joint_id),
          controlMode=self._pybullet_client.VELOCITY_CONTROL,
          targetVelocity=0,
          force=0)
    for i, jointId in enumerate(self._joint_ids):
      angle = self._robot_config.INIT_MOTOR_ANGLES[i] + self._robot_config.JOINT_OFFSETS[i]
      self._pybullet_client.resetJointState(
          self.quadruped, jointId, angle, targetVelocity=0)

  ######################################################################################
  # URDF related
  ######################################################################################
  def _LoadRobotURDF(self):
    """Loads the URDF file for the robot."""
    urdf_file =  os.path.join(self._urdf_root, self._robot_config.URDF_FILENAME)
    if self._self_collision_enabled:
      self.quadruped = self._pybullet_client.loadURDF(
          urdf_file,
          self._GetDefaultInitPosition(),
          self._GetDefaultInitOrientation(),
          flags=self._pybullet_client.URDF_USE_SELF_COLLISION)
    else:
      self.quadruped = self._pybullet_client.loadURDF(
          urdf_file, self._GetDefaultInitPosition(),
          self._GetDefaultInitOrientation())

  def _BuildUrdfIds(self):
    """Build the link Ids from its name in the URDF file.

    Raises:
      ValueError: Unknown category of the joint name.
    """
    num_joints = self._pybullet_client.getNumJoints(self.quadruped)
    self._chassis_link_ids = [-1] # just base link
    self._leg_link_ids = []   # all leg links (hip, thigh, calf)
    self._motor_link_ids = [] # all leg links (hip, thigh, calf)

    self._joint_ids=[]      # all motor joints
    self._hip_ids = []      # hip joint indices only
    self._thigh_ids = []    # thigh joint indices only
    self._calf_ids = []     # calf joint indices only
    self._foot_link_ids = [] # foot joint indices

    for i in range(num_joints):
      joint_info = self._pybullet_client.getJointInfo(self.quadruped, i)
      # print(joint_info)
      joint_name = joint_info[1].decode("UTF-8")
      joint_id = self._joint_name_to_id[joint_name]
      if self._robot_config._CHASSIS_NAME_PATTERN.match(joint_name):
        self._chassis_link_ids = [joint_id] #.append(joint_id) # _should_ be only one!
      elif self._robot_config._HIP_NAME_PATTERN.match(joint_name):
        self._hip_ids.append(joint_id)
      elif self._robot_config._THIGH_NAME_PATTERN.match(joint_name):
        self._thigh_ids.append(joint_id)
      elif self._robot_config._CALF_NAME_PATTERN.match(joint_name):
        self._calf_ids.append(joint_id)
      elif self._robot_config._FOOT_NAME_PATTERN.match(joint_name):
        self._foot_link_ids.append(joint_id)
      else:
        continue
        raise ValueError("Unknown category of joint %s" % joint_name)


    # print('chassis',self._chassis_link_ids)
    # for i in range(-1,num_joints):
    #   print(self._pybullet_client.getDynamicsInfo(self.quadruped, i))

    # everything associated with the leg links
    self._joint_ids.extend(self._hip_ids)
    self._joint_ids.extend(self._thigh_ids)
    self._joint_ids.extend(self._calf_ids)
    # sort in case any weird order
    self._joint_ids.sort()
    self._hip_ids.sort()
    self._thigh_ids.sort()
    self._calf_ids.sort()
    self._foot_link_ids.sort()

    # print('joint ids', self._joint_ids)
    # print('hip ids', self._hip_ids)
    # print('_thigh_ids', self._thigh_ids)
    # print('_calf_ids', self._calf_ids)
    # print('_foot_link_ids', self._foot_link_ids)

  def _RecordMassAndInertiaInfoFromURDF(self):
    """Records the mass information from the URDF file.
    Divide into base, leg, foot, and total (all links)
    """
    # base 
    self._base_mass_urdf = []
    self._base_inertia_urdf = []

    for chassis_id in self._chassis_link_ids:
      self._base_mass_urdf.append(
          self._pybullet_client.getDynamicsInfo(self.quadruped, chassis_id)[0])
      self._base_inertia_urdf.append(
          self._pybullet_client.getDynamicsInfo(self.quadruped, chassis_id)[2])

    # leg masses
    self._leg_masses_urdf = []
    self._leg_inertia_urdf = []
    for leg_id in self._joint_ids:
      self._leg_masses_urdf.append(
          self._pybullet_client.getDynamicsInfo(self.quadruped, leg_id)[0])
      self._leg_inertia_urdf.append(
          self._pybullet_client.getDynamicsInfo(self.quadruped, leg_id)[2])

    # foot masses
    self._foot_masses_urdf = []
    self._foot_inertia_urdf = []
    for foot_id in self._foot_link_ids:
      self._foot_masses_urdf.append(
          self._pybullet_client.getDynamicsInfo(self.quadruped, foot_id)[0])
      self._foot_inertia_urdf.append(
          self._pybullet_client.getDynamicsInfo(self.quadruped, foot_id)[2])


    # all masses present in URDF
    self._total_mass_urdf = []
    self._total_inertia_urdf = []
    # don't forget base (which is at -1)
    for j in range(-1,self._pybullet_client.getNumJoints(self.quadruped)):
      self._total_mass_urdf.append(
          self._pybullet_client.getDynamicsInfo(self.quadruped,j)[0])
      self._total_inertia_urdf.append(
          self._pybullet_client.getDynamicsInfo(self.quadruped,j)[2])

    # set originals as needed
    self._original_base_mass_urdf = tuple(self._base_mass_urdf)
    self._original_leg_masses_urdf = tuple(self._leg_masses_urdf)
    self._original_foot_masses_urdf = tuple(self._foot_masses_urdf)
    self._original_total_mass_urdf = tuple(self._total_mass_urdf)

    self._original_base_inertia_urdf = tuple(self._base_inertia_urdf)
    self._original_leg_inertia_urdf = tuple(self._leg_inertia_urdf)
    self._original_foot_inertia_urdf = tuple(self._foot_inertia_urdf)
    self._original_total_inertia_urdf = tuple(self._total_inertia_urdf)

    #print('total mass', sum(self._total_mass_urdf))

  def _BuildJointNameToIdDict(self):
    """_BuildJointNameToIdDict """
    num_joints = self._pybullet_client.getNumJoints(self.quadruped)
    self._joint_name_to_id = {}
    for i in range(num_joints):
      joint_info = self._pybullet_client.getJointInfo(self.quadruped, i)
      self._joint_name_to_id[joint_info[1].decode("UTF-8")] = joint_info[0]

  def _BuildMotorIdList(self):
    self._motor_id_list = self._joint_ids

  def _RemoveDefaultJointDamping(self):
    """Pybullet convention/necessity  """
    num_joints = self._pybullet_client.getNumJoints(self.quadruped)
    for i in range(num_joints):
      joint_info = self._pybullet_client.getJointInfo(self.quadruped, i)
      self._pybullet_client.changeDynamics(
          joint_info[0], -1, linearDamping=0, angularDamping=0)

  def _SetLateralFriction(self,lateral_friction=1.0):
    """ Lateral friction to be set for every link. 
    NOTE: pybullet default is 0.5 - so call to set to 1 as default
    """
    num_joints = self._pybullet_client.getNumJoints(self.quadruped)
    for i in range(-1,num_joints):
      self._pybullet_client.changeDynamics(self.quadruped, i, lateralFriction=lateral_friction)

  def _SetMaxJointVelocities(self):
    """Set maximum joint velocities from robot_config, the pybullet default is 100 rad/s """
    for i, link_id in enumerate(self._joint_ids):
      self._pybullet_client.changeDynamics(self.quadruped, link_id, maxJointVelocity=self._robot_config.VELOCITY_LIMITS[i]) 

  def GetBaseMassFromURDF(self):
    """Get the mass of the base from the URDF file."""
    return self._base_mass_urdf

  def GetLegMassesFromURDF(self):
    """Get the mass of the legs from the URDF file."""
    return self._leg_masses_urdf

  def GetFootMassesFromURDF(self):
    """Get the mass of the feet from the URDF file."""
    return self._foot_masses_urdf

  def GetTotalMassFromURDF(self):
    """Get the total mass from all links in the URDF."""
    return self._total_mass_urdf

In [ ]:
# SPDX-FileCopyrightText: Copyright (c) 2022 Guillaume Bellegarda. All rights reserved.
# SPDX-License-Identifier: BSD-3-Clause
# 
# Redistribution and use in source and binary forms, with or without
# modification, are permitted provided that the following conditions are met:
#
# 1. Redistributions of source code must retain the above copyright notice, this
# list of conditions and the following disclaimer.
#
# 2. Redistributions in binary form must reproduce the above copyright notice,
# this list of conditions and the following disclaimer in the documentation
# and/or other materials provided with the distribution.
#
# 3. Neither the name of the copyright holder nor the names of its
# contributors may be used to endorse or promote products derived from
# this software without specific prior written permission.
#
# THIS SOFTWARE IS PROVIDED BY THE COPYRIGHT HOLDERS AND CONTRIBUTORS "AS IS"
# AND ANY EXPRESS OR IMPLIED WARRANTIES, INCLUDING, BUT NOT LIMITED TO, THE
# IMPLIED WARRANTIES OF MERCHANTABILITY AND FITNESS FOR A PARTICULAR PURPOSE ARE
# DISCLAIMED. IN NO EVENT SHALL THE COPYRIGHT HOLDER OR CONTRIBUTORS BE LIABLE
# FOR ANY DIRECT, INDIRECT, INCIDENTAL, SPECIAL, EXEMPLARY, OR CONSEQUENTIAL
# DAMAGES (INCLUDING, BUT NOT LIMITED TO, PROCUREMENT OF SUBSTITUTE GOODS OR
# SERVICES; LOSS OF USE, DATA, OR PROFITS; OR BUSINESS INTERRUPTION) HOWEVER
# CAUSED AND ON ANY THEORY OF LIABILITY, WHETHER IN CONTRACT, STRICT LIABILITY,
# OR TORT (INCLUDING NEGLIGENCE OR OTHERWISE) ARISING IN ANY WAY OUT OF THE USE
# OF THIS SOFTWARE, EVEN IF ADVISED OF THE POSSIBILITY OF SUCH DAMAGE.
#
# Copyright (c) 2022 EPFL, Guillaume Bellegarda

"""Defines the A1 robot related constants and URDF specs."""
import numpy as np
import re
import pybullet as pyb
import os, inspect

currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)

URDF_ROOT = parentdir 
URDF_FILENAME = "a1_description/urdf/a1.urdf"

##################################################################################
# Default robot configuration (i.e. base and joint positions, etc.)
##################################################################################
NUM_MOTORS = 12
NUM_LEGS = 4
MOTORS_PER_LEG = 3

INIT_RACK_POSITION = [0, 0, 1] # when hung up in air (for debugging)
INIT_POSITION = [0, 0, 0.305]  # normal initial height
IS_FALLEN_HEIGHT = 0.18        # height at which robot is considered fallen

INIT_ORIENTATION = (0, 0, 0, 1) 
_, INIT_ORIENTATION_INV = pyb.invertTransform(
        position=[0, 0, 0], orientation=INIT_ORIENTATION)

# default angles (for init)
DEFAULT_HIP_ANGLE = 0
DEFAULT_THIGH_ANGLE = np.pi/4 
DEFAULT_CALF_ANGLE = -np.pi/2 
INIT_JOINT_ANGLES = np.array([  DEFAULT_HIP_ANGLE, 
                                DEFAULT_THIGH_ANGLE, 
                                DEFAULT_CALF_ANGLE] * NUM_LEGS)
INIT_MOTOR_ANGLES = INIT_JOINT_ANGLES
# Used to convert the robot SDK joint angles to URDF joint angles.
JOINT_DIRECTIONS = np.array([1, 1, 1, 1, 1, 1, 
                             1, 1, 1, 1, 1, 1])

# joint offsets 
HIP_JOINT_OFFSET = 0.0
THIGH_JOINT_OFFSET = 0.0
CALF_JOINT_OFFSET = 0.0

# Used to convert the robot SDK joint angles to URDF joint angles.
JOINT_OFFSETS = np.array([  HIP_JOINT_OFFSET, 
                            THIGH_JOINT_OFFSET,
                            CALF_JOINT_OFFSET] * NUM_LEGS)

# Kinematics
HIP_LINK_LENGTH = 0.0838
THIGH_LINK_LENGTH = 0.2
CALF_LINK_LENGTH = 0.2

NOMINAL_FOOT_POS_LEG_FRAME = np.array([ 0, -HIP_LINK_LENGTH, -0.25,
                                        0,  HIP_LINK_LENGTH, -0.25,
                                        0, -HIP_LINK_LENGTH, -0.25,
                                        0,  HIP_LINK_LENGTH, -0.25])

##################################################################################
# Actuation limits/gains, position, and velocity limits
##################################################################################
# joint limits on real system
UPPER_ANGLE_JOINT = np.array([ 0.802851455917,  4.18879020479, -0.916297857297 ] * NUM_LEGS)
LOWER_ANGLE_JOINT = np.array([-0.802851455917, -1.0471975512 , -2.69653369433  ] * NUM_LEGS)

# modified range in simulation (min observation space for RL)
UPPER_ANGLE_JOINT = np.array([ 0.2,  DEFAULT_THIGH_ANGLE + 0.4, DEFAULT_CALF_ANGLE + 0.4 ] * NUM_LEGS)
LOWER_ANGLE_JOINT = np.array([-0.2,  DEFAULT_THIGH_ANGLE - 0.4, DEFAULT_CALF_ANGLE - 0.4 ] * NUM_LEGS)

# torque and velocity limits 
TORQUE_LIMITS   = np.asarray( [33.5] * NUM_MOTORS )
VELOCITY_LIMITS = np.asarray( [21.0] * NUM_MOTORS ) 

# Sample Joint Gains
MOTOR_KP = [100.0, 100.0, 100.0] * NUM_LEGS
MOTOR_KD = [2.0, 2.0, 2.0] * NUM_LEGS

# MOTOR_KP = [55,55,55] * NUM_LEGS
# MOTOR_KD = [0.8,0.8,0.8] * NUM_LEGS

# Sample Cartesian Gains
kpCartesian = np.diag([500,500,500])
kdCartesian = np.diag([10,10,10])

kpCartesian = np.diag([700,700,700])
kdCartesian = np.diag([12,12,12])

##################################################################################
# Hip, thigh, calf strings, naming conventions from URDF (don't modify)
##################################################################################
JOINT_NAMES = (
    # front right leg
    "FR_hip_joint", 
    "FR_thigh_joint", 
    "FR_calf_joint",
    # front left leg
    "FL_hip_joint", 
    "FL_thigh_joint", 
    "FL_calf_joint",
    # rear right leg
    "RR_hip_joint", 
    "RR_thigh_joint", 
    "RR_calf_joint",
    # rear left leg
    "RL_hip_joint", 
    "RL_thigh_joint", 
    "RL_calf_joint",
)
MOTOR_NAMES = JOINT_NAMES

# standard across all robots
_CHASSIS_NAME_PATTERN = re.compile(r"\w*floating_base\w*")
_HIP_NAME_PATTERN = re.compile(r"\w+_hip_j\w+")
_THIGH_NAME_PATTERN = re.compile(r"\w+_thigh_j\w+")
_CALF_NAME_PATTERN = re.compile(r"\w+_calf_j\w+")
_FOOT_NAME_PATTERN = re.compile(r"\w+_foot_\w+")


In [ ]:
# SPDX-FileCopyrightText: Copyright (c) 2022 Guillaume Bellegarda. All rights reserved.
# SPDX-License-Identifier: BSD-3-Clause
# 
# Redistribution and use in source and binary forms, with or without
# modification, are permitted provided that the following conditions are met:
#
# 1. Redistributions of source code must retain the above copyright notice, this
# list of conditions and the following disclaimer.
#
# 2. Redistributions in binary form must reproduce the above copyright notice,
# this list of conditions and the following disclaimer in the documentation
# and/or other materials provided with the distribution.
#
# 3. Neither the name of the copyright holder nor the names of its
# contributors may be used to endorse or promote products derived from
# this software without specific prior written permission.
#
# THIS SOFTWARE IS PROVIDED BY THE COPYRIGHT HOLDERS AND CONTRIBUTORS "AS IS"
# AND ANY EXPRESS OR IMPLIED WARRANTIES, INCLUDING, BUT NOT LIMITED TO, THE
# IMPLIED WARRANTIES OF MERCHANTABILITY AND FITNESS FOR A PARTICULAR PURPOSE ARE
# DISCLAIMED. IN NO EVENT SHALL THE COPYRIGHT HOLDER OR CONTRIBUTORS BE LIABLE
# FOR ANY DIRECT, INDIRECT, INCIDENTAL, SPECIAL, EXEMPLARY, OR CONSEQUENTIAL
# DAMAGES (INCLUDING, BUT NOT LIMITED TO, PROCUREMENT OF SUBSTITUTE GOODS OR
# SERVICES; LOSS OF USE, DATA, OR PROFITS; OR BUSINESS INTERRUPTION) HOWEVER
# CAUSED AND ON ANY THEORY OF LIABILITY, WHETHER IN CONTRACT, STRICT LIABILITY,
# OR TORT (INCLUDING NEGLIGENCE OR OTHERWISE) ARISING IN ANY WAY OUT OF THE USE
# OF THIS SOFTWARE, EVEN IF ADVISED OF THE POSSIBILITY OF SUCH DAMAGE.
#
# Copyright (c) 2022 EPFL, Guillaume Bellegarda

"""This file implements the gym environment for a quadruped. """
import os, inspect
# so we can import files
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
os.sys.path.insert(0, currentdir)

import time, datetime
import numpy as np
# gym
import gym
from gym import spaces
from gym.utils import seeding
# pybullet
import pybullet
import pybullet_utils.bullet_client as bc
import pybullet_data
import random
random.seed(10)
# quadruped and configs
import quadruped_colab
import configs_a1 as robot_config
from hopf_network import HopfNetwork


ACTION_EPS = 0.01
OBSERVATION_EPS = 0.01
VIDEO_LOG_DIRECTORY = 'videos/' + datetime.datetime.now().strftime("vid-%Y-%m-%d-%H-%M-%S-%f")

# Implemented observation spaces for deep reinforcement learning: 
#   "DEFAULT":    motor angles and velocities, body orientation
#   "LR_COURSE_OBS":  [TODO: what should you include? what is reasonable to measure on the real system? CPG states?] 

# Tasks to be learned with reinforcement learning
#     - "FWD_LOCOMOTION"
#         reward forward progress only
#     - "LR_COURSE_TASK" 
#         [TODO: what should you train for?]
#         Ideally we want to command A1 to run in any direction while expending minimal energy
#         How will you construct your reward function? 

# Motor control modes:
#   - "TORQUE": 
#         supply raw torques to each motor (12)
#   - "PD": 
#         supply desired joint positions to each motor (12)
#         torques are computed based on the joint position/velocity error
#   - "CARTESIAN_PD": 
#         supply desired foot positions for each leg (12)
#         torques are computed based on the foot position/velocity error
#   - "CPG": 
#         supply desired CPG state modulations (8), mapped to foot positions
#         torques are computed based on inverse kinematics + joint PD (or you can add Cartesian PD)


EPISODE_LENGTH = 10   # how long before we reset the environment (max episode length for RL)
MAX_FWD_VELOCITY = 1  # to avoid exploiting simulator dynamics, cap max reward for body velocity 

# CPG quantities
MU_LOW = 1
MU_UPP = 2


class QuadrupedGymEnv(gym.Env):
  """The gym environment for a quadruped {Unitree A1}.

  It simulates the locomotion of a quadrupedal robot. 
  The state space, action space, and reward functions can be chosen with:
  observation_space_mode, motor_control_mode, task_env.
  """
  def __init__(
      self,
      robot_config=robot_config,
      isRLGymInterface=True,
      time_step=0.001,
      action_repeat=10,  
      distance_weight=2,
      energy_weight=0.008,
      motor_control_mode="CPG",
      task_env="FWD_LOCOMOTION",
      observation_space_mode="LR_COURSE_OBS",
      on_rack=False,
      render=False,
      record_video=False,
      add_noise=True,
      test_env=False,
      competition_env=False, # NOT ALLOWED FOR TRAINING!
      **kwargs): # any extra arguments from legacy
    """Initialize the quadruped gym environment.

    Args:
      robot_config: The robot config file, contains A1 parameters.
      isRLGymInterface: If the gym environment is being run as RL or not. Affects
        if the actions should be scaled.
      time_step: Simulation time step.
      action_repeat: The number of simulation steps where the same actions are applied.
      distance_weight: The weight of the distance term in the reward.
      energy_weight: The weight of the energy term in the reward.
      motor_control_mode: Whether to use torque control, PD, control, CPG, etc.
      task_env: Task trying to learn (fwd locomotion, standup, etc.)
      observation_space_mode: what should be in here? Check available functions in quadruped.py
        also consider CPG states (amplitudes/phases)
      on_rack: Whether to place the quadruped on rack. This is only used to debug
        the walking gait. In this mode, the quadruped's base is hanged midair so
        that its walking gait is clearer to visualize.
      render: Whether to render the simulation.
      record_video: Whether to record a video of each trial.
      add_noise: vary coefficient of friction
      test_env: add random terrain 
      competition_env: course competition block format, fixed coefficient of friction 
    """
    self._robot_config = robot_config
    self._isRLGymInterface = isRLGymInterface
    self._time_step = time_step
    self._action_repeat = action_repeat
    self._distance_weight = distance_weight
    self._energy_weight = energy_weight
    self._motor_control_mode = motor_control_mode
    self._TASK_ENV = task_env
    self._observation_space_mode = observation_space_mode
    self._hard_reset = True # must fully reset simulation at init
    self._on_rack = on_rack
    self._is_render = render
    self._is_record_video = record_video
    self._add_noise = add_noise
    self._using_test_env = test_env
    self._using_competition_env = competition_env
    if competition_env:
      test_env = False
      self._using_test_env = False
      self._add_noise = False
    if test_env:
      self._add_noise = True
      self._observation_noise_stdev = 0.01 #
    else:
      self._observation_noise_stdev = 0.0

    # other bookkeeping 
    self._num_bullet_solver_iterations = int(300 / action_repeat) 
    self._env_step_counter = 0
    self._sim_step_counter = 0
    self._last_base_position = [0, 0, 0]
    self._last_frame_time = 0.0 # for rendering 
    self._MAX_EP_LEN = EPISODE_LENGTH # max sim time in seconds, arbitrary
    self._action_bound = 1.0

    # if using CPG
    self.setupCPG()

    self.setupActionSpace()
    self.setupObservationSpace()
    if self._is_render:
      self._pybullet_client = bc.BulletClient(connection_mode=pybullet.GUI)
    else:
      self._pybullet_client = bc.BulletClient()
    self._configure_visualizer()

    self.videoLogID = None
    self.seed()
    self.reset()
 
  def setupCPG(self):
    self._cpg = HopfNetwork(use_RL=True)

  ######################################################################################
  # RL Observation and Action spaces 
  ######################################################################################
  def setupObservationSpace(self):
    """Set up observation space for RL. """
    if self._observation_space_mode == "DEFAULT":
      observation_high = (np.concatenate((self._robot_config.UPPER_ANGLE_JOINT*0.9,
                                         self._robot_config.VELOCITY_LIMITS,
                                         np.array([1.0]*4))) +  OBSERVATION_EPS)
      observation_low = (np.concatenate((self._robot_config.LOWER_ANGLE_JOINT,
                                         -self._robot_config.VELOCITY_LIMITS,
                                         np.array([-1.0]*4))) -  OBSERVATION_EPS)
    elif self._observation_space_mode == "LR_COURSE_OBS":
      # [TODO] Set observation upper and lower ranges. What are reasonable limits? 
      # Note 50 is arbitrary below, you may have more or less
      # if using CPG-RL, remember to include limits on these

      observation_high = (np.concatenate((np.array([ 0.261799,  1.5708, -0.916297857297 ] * self._robot_config.NUM_LEGS), # joint limit
                                         self._robot_config.VELOCITY_LIMITS,
                                         np.array([ 2 ] * self._robot_config.NUM_LEGS),
                                         np.array([ 2*np.pi ] * self._robot_config.NUM_LEGS),
                                         np.array([1.0]*4))) +  OBSERVATION_EPS)
      observation_low = (np.concatenate((np.array([ -0.261799,  0.261799, -2.69653369433 ] * self._robot_config.NUM_LEGS), # joint limit
                                         -self._robot_config.VELOCITY_LIMITS/1.5,
                                         np.array([ 0, ] * self._robot_config.NUM_LEGS),
                                         np.array([ 0 ] * self._robot_config.NUM_LEGS),
                                         np.array([-1.0]*4))) -  OBSERVATION_EPS)
    else:
      raise ValueError("observation space not defined or not intended")

    self.observation_space = spaces.Box(observation_low, observation_high, dtype=np.float32)

  def setupActionSpace(self):
    """ Set up action space for RL. """
    if self._motor_control_mode in ["PD","TORQUE", "CARTESIAN_PD"]:
      action_dim = 12
    elif self._motor_control_mode in ["CPG"]:
      action_dim = 8
    else:
      raise ValueError("motor control mode " + self._motor_control_mode + " not implemented yet.")
    action_high = np.array([1] * action_dim)
    self.action_space = spaces.Box(-action_high, action_high, dtype=np.float32)
    self._action_dim = action_dim


  def _get_observation(self):
    """Get observation, depending on obs space selected. """
    if self._observation_space_mode == "DEFAULT":
      self._observation = np.concatenate((self.robot.GetMotorAngles(), 
                                          self.robot.GetMotorVelocities(),
                                          self.robot.GetBaseOrientation() ))
    elif self._observation_space_mode == "LR_COURSE_OBS":
      # [TODO] Get observation from robot. What are reasonable measurements we could get on hardware?
      # if using the CPG, you can include states with self._cpg.get_r(), for example
      # 50 is arbitrary
      self._observation = np.concatenate((self.robot.GetMotorAngles(), 
                                          self.robot.GetMotorVelocities(),
                                          self._cpg.get_r(),
                                          self._cpg.get_theta(),
                                          self.robot.GetBaseOrientation() ))

    else:
      raise ValueError("observation space not defined or not intended")

    self._add_obs_noise = (np.random.normal(scale=self._observation_noise_stdev, size=self._observation.shape) *
          self.observation_space.high)
    return self._observation

  def _noisy_observation(self):
    self._get_observation()
    observation = np.array(self._observation)
    if self._observation_noise_stdev > 0:
      observation += self._add_obs_noise
    return observation

  ######################################################################################
  # Termination and reward
  ######################################################################################
  def is_fallen(self,dot_prod_min=0.85):
    """Decide whether the quadruped has fallen.

    If the up directions between the base and the world is larger (the dot
    product is smaller than 0.85) or the base is very low on the ground
    (the height is smaller than 0.13 meter), the quadruped is considered fallen.

    Returns:
      Boolean value that indicates whether the quadruped has fallen.
    """
    base_rpy = self.robot.GetBaseOrientationRollPitchYaw()
    orientation = self.robot.GetBaseOrientation()
    rot_mat = self._pybullet_client.getMatrixFromQuaternion(orientation)
    local_up = rot_mat[6:]
    pos = self.robot.GetBasePosition()
    return (np.dot(np.asarray([0, 0, 1]), np.asarray(local_up)) < dot_prod_min or pos[2] < self._robot_config.IS_FALLEN_HEIGHT)

  def _termination(self):
    """Decide whether we should stop the episode and reset the environment. """
    return self.is_fallen() 

  def _reward_fwd_locomotion(self, des_vel_x=0.5):
    """Learn forward locomotion at a desired velocity. """
    # track the desired velocity 
    vel_tracking_reward = 0.05 * np.exp( -1/ 0.25 *  (self.robot.GetBaseLinearVelocity()[0] - des_vel_x)**2 )
    # minimize yaw (go straight)
    yaw_reward = -0.2 * np.abs(self.robot.GetBaseOrientationRollPitchYaw()[2]) 
    # don't drift laterally 
    drift_reward = -0.01 * abs(self.robot.GetBasePosition()[1]) 
    # minimize energy 
    energy_reward = 0 
    for tau,vel in zip(self._dt_motor_torques,self._dt_motor_velocities):
      energy_reward += np.abs(np.dot(tau,vel)) * self._time_step

    reward = vel_tracking_reward \
            + yaw_reward \
            + drift_reward \
            - 0.01 * energy_reward \
            - 0.1 * np.linalg.norm(self.robot.GetBaseOrientation() - np.array([0,0,0,1]))

    return max(reward,0) # keep rewards positive


  def _reward_lr_course(self):
    """ Implement your reward function here. How will you improve upon the above? """
    # [TODO] add your reward function. 
    return 0

  def _reward(self):
    """ Get reward depending on task"""
    if self._TASK_ENV == "FWD_LOCOMOTION":
      return self._reward_fwd_locomotion()
    elif self._TASK_ENV == "LR_COURSE_TASK":
      return self._reward_lr_course()
    else:
      raise ValueError("This task mode not implemented yet.")

  ######################################################################################
  # Step simulation, map policy network actions to joint commands, etc. 
  ######################################################################################
  def _transform_action_to_motor_command(self, action):
    """ Map actions from RL (i.e. in [-1,1]) to joint commands based on motor_control_mode. """
    # clip actions to action bounds
    action = np.clip(action, -self._action_bound - ACTION_EPS,self._action_bound + ACTION_EPS)
    if self._motor_control_mode == "PD":
      action = self._scale_helper(action, self._robot_config.LOWER_ANGLE_JOINT, self._robot_config.UPPER_ANGLE_JOINT)
      action = np.clip(action, self._robot_config.LOWER_ANGLE_JOINT, self._robot_config.UPPER_ANGLE_JOINT)
    elif self._motor_control_mode == "CARTESIAN_PD":
      action = self.ScaleActionToCartesianPos(action)
    elif self._motor_control_mode == "CPG":
      action = self.ScaleActionToCPGStateModulations(action)
    else:
      raise ValueError("RL motor control mode" + self._motor_control_mode + "not implemented yet.")
    return action

  def _scale_helper(self, action, lower_lim, upper_lim):
    """Helper to linearly scale from [-1,1] to lower/upper limits. """
    new_a = lower_lim + 0.5 * (action + 1) * (upper_lim - lower_lim)
    return np.clip(new_a, lower_lim, upper_lim)

  def ScaleActionToCartesianPos(self,actions):
    """Scale RL action to Cartesian PD ranges. 
    Edit ranges, limits etc., but make sure to use Cartesian PD to compute the torques. 
    """
    # clip RL actions to be between -1 and 1 (standard RL technique)
    u = np.clip(actions,-1,1)
    # scale to corresponding desired foot positions (i.e. ranges in x,y,z we allow the agent to choose foot positions)
    # [TODO: edit (do you think these should these be increased? How limiting is this?)]
    scale_array = np.array([0.1, 0.05, 0.08]*4)
    # add to nominal foot position in leg frame (what are the final ranges?)
    des_foot_pos = self._robot_config.NOMINAL_FOOT_POS_LEG_FRAME + scale_array*u

    # get Cartesian kp and kd gains (can be modified)
    kpCartesian = self._robot_config.kpCartesian
    kdCartesian = self._robot_config.kdCartesian
    # get current motor velocities
    dq = self.robot.GetMotorVelocities()
    des_joint_vel = np.zeros(3)

    action = np.zeros(12)
    for i in range(4):
      # Get current Jacobian and foot position in leg frame (see ComputeJacobianAndPosition() in quadruped.py)
      # [TODO] 
      J, p = self.robot.ComputeJacobianAndPosition(i)
      # Get current foot velocity in leg frame (Equation 2)
      # [TODO] 
      v = J @ dq[i*3:i*3+3]
      des_v = J @ des_joint_vel
      # Calculate torque contribution from Cartesian PD (Equation 5) [Make sure you are using matrix multiplications]
      # tau += np.zeros(3) # [TODO]
      tau += np.transpose(J) @ (kpCartesian@(des_foot_pos-p) + kdCartesian@(des_v - v))
      action[3*i:3*i+3] = tau

    return action

  def ScaleActionToCPGStateModulations(self,actions):
    """Scale RL action to CPG modulation parameters."""
    # clip RL actions to be between -1 and 1 (standard RL technique)
    u = np.clip(actions,-1,1)

    # scale omega to ranges, and set in CPG (range is an example)
    omega = self._scale_helper( u[0:4], 5, 5*2*np.pi)
    self._cpg.set_omega_rl(omega)

    # scale mu to ranges, and set in CPG (squared since we converge to the sqrt in the CPG amplitude)
    mus = self._scale_helper( u[4:8], MU_LOW**2, MU_UPP**2)
    self._cpg.set_mu_rl(mus)

    # integrate CPG, get mapping to foot positions
    xs,zs = self._cpg.update()

    # IK parameters
    foot_y = self._robot_config.HIP_LINK_LENGTH
    sideSign = np.array([-1, 1, -1, 1]) # get correct hip sign (body right is negative)
    # get motor kp and kd gains (can be modified)
    kp = self._robot_config.MOTOR_KP # careful of size!
    kd = self._robot_config.MOTOR_KD
    # get current motor velocities
    q = self.robot.GetMotorAngles()
    dq = self.robot.GetMotorVelocities()
    des_joint_vel = np.zeros(3)

    action = np.zeros(12)
    # loop through each leg
    for i in range(4):
      # initialize torques for legi
      tau = np.zeros(3)
      # get desired foot i pos (xi, yi, zi) in leg frame
      leg_xyz = np.array([xs[i],sideSign[i] * foot_y,zs[i]])
      # call inverse kinematics to get corresponding joint angles (see ComputeInverseKinematics() in quadruped.py)
      # leg_q = np.zeros(3) # [TODO] 
      leg_q = self.robot.ComputeInverseKinematics(i ,leg_xyz)
      # Add joint PD contribution to tau for leg i (Equation 4)
      # tau += np.zeros(3) # [TODO] 
      tau = kp[i*3:i*3+3]*(leg_q-q[i*3:i*3+3]) + kd[i*3:i*3+3]*(des_joint_vel - dq[i*3:i*3+3])

      # add Cartesian PD contribution (as you wish) ?????????????????????????????????????????
      # tau += self.ScaleActionToCartesianPos(actions)

      action[3*i:3*i+3] = tau

    return action


  def step(self, action):
    """ Step forward the simulation, given the action. """
    curr_act = action.copy()
    # save motor torques and velocities to compute power in reward function
    self._dt_motor_torques = []
    self._dt_motor_velocities = []
    
    for _ in range(self._action_repeat):
      if self._isRLGymInterface: 
        proc_action = self._transform_action_to_motor_command(curr_act)
      else:
        proc_action = curr_act 
      self.robot.ApplyAction(proc_action)
      self._pybullet_client.stepSimulation()
      self._sim_step_counter += 1
      self._dt_motor_torques.append(self.robot.GetMotorTorques())
      self._dt_motor_velocities.append(self.robot.GetMotorVelocities())

      if self._is_render:
        self._render_step_helper()

    self._last_action = curr_act
    self._env_step_counter += 1
    reward = self._reward()
    done = False
    if self._termination() or self.get_sim_time() > self._MAX_EP_LEN:
      done = True

    return np.array(self._noisy_observation()), reward, done, {'base_pos': self.robot.GetBasePosition()} 

  ######################################################################################
  # Reset
  ######################################################################################
  def reset(self):
    """ Set up simulation environment. """
    mu_min = 0.5
    if self._hard_reset:
      # set up pybullet simulation
      self._pybullet_client.resetSimulation()
      self._pybullet_client.setPhysicsEngineParameter(
          numSolverIterations=int(self._num_bullet_solver_iterations))
      self._pybullet_client.setTimeStep(self._time_step)
      self.plane = self._pybullet_client.loadURDF(pybullet_data.getDataPath()+"/plane.urdf", 
                                                  basePosition=[80,0,0]) # to extend available running space (shift)
      self._pybullet_client.changeVisualShape(self.plane, -1, rgbaColor=[1, 1, 1, 0.9])
      self._pybullet_client.configureDebugVisualizer(
          self._pybullet_client.COV_ENABLE_PLANAR_REFLECTION, 0)
      self._pybullet_client.setGravity(0, 0, -9.8)
      self.robot = (quadruped.Quadruped(pybullet_client=self._pybullet_client,
                                         robot_config=self._robot_config,
                                         motor_control_mode=self._motor_control_mode,
                                         on_rack=self._on_rack,
                                         render=self._is_render))
      if self._using_competition_env:
        self._ground_mu_k = ground_mu_k = 0.8
        self._pybullet_client.changeDynamics(self.plane, -1, lateralFriction=ground_mu_k)
        self.add_competition_blocks()
        self._add_noise = False # double check
        self._using_test_env = False # double check 

      if self._add_noise:
        ground_mu_k = mu_min+(1-mu_min)*np.random.random()
        self._ground_mu_k = ground_mu_k
        self._pybullet_client.changeDynamics(self.plane, -1, lateralFriction=ground_mu_k)
        if self._is_render:
          print('ground friction coefficient is', ground_mu_k)

      if self._using_test_env:
        self.add_random_boxes()
        self._add_base_mass_offset()

    else:
      self.robot.Reset(reload_urdf=False)

    self.setupCPG()
    self._env_step_counter = 0
    self._sim_step_counter = 0
    self._last_base_position = [0, 0, 0]

    if self._is_render:
      self._pybullet_client.resetDebugVisualizerCamera(self._cam_dist, self._cam_yaw,
                                                       self._cam_pitch, [0, 0, 0])

    self._settle_robot()
    self._last_action = np.zeros(self._action_dim)
    if self._is_record_video:
      self.recordVideoHelper()
    return self._noisy_observation()


  def _settle_robot(self):
    """ Settle robot and add noise to init configuration. """
    # change to PD control mode to set initial position, then set back..
    tmp_save_motor_control_mode_ENV = self._motor_control_mode
    tmp_save_motor_control_mode_ROB = self.robot._motor_control_mode
    self._motor_control_mode = "PD"
    self.robot._motor_control_mode = "PD"
    try:
      tmp_save_motor_control_mode_MOT = self.robot._motor_model._motor_control_mode
      self.robot._motor_model._motor_control_mode = "PD"
    except:
      pass
    init_motor_angles = self._robot_config.INIT_MOTOR_ANGLES + self._robot_config.JOINT_OFFSETS
    #if self._is_render:
      #time.sleep(0.2)
    for _ in range(1000):
      self.robot.ApplyAction(init_motor_angles)
      #if self._is_render:
        #time.sleep(0.001)
      self._pybullet_client.stepSimulation()
    
    # set control mode back
    self._motor_control_mode = tmp_save_motor_control_mode_ENV
    self.robot._motor_control_mode = tmp_save_motor_control_mode_ROB
    try:
      self.robot._motor_model._motor_control_mode = tmp_save_motor_control_mode_MOT
    except:
      pass

  ######################################################################################
  # Render, record videos, bookkeping, and misc pybullet helpers.  
  ######################################################################################
  def startRecordingVideo(self,name):
    self.videoLogID = self._pybullet_client.startStateLogging(
                            self._pybullet_client.STATE_LOGGING_VIDEO_MP4, 
                            name)

  def stopRecordingVideo(self):
    self._pybullet_client.stopStateLogging(self.videoLogID)

  def close(self):
    if self._is_record_video:
      self.stopRecordingVideo()
    self._pybullet_client.disconnect()

  def recordVideoHelper(self, extra_filename=None):
    """ Helper to record video, if not already, or end and start a new one """
    # If no ID, this is the first video, so make a directory and start logging
    if self.videoLogID == None:
      directoryName = VIDEO_LOG_DIRECTORY
      assert isinstance(directoryName, str)
      os.makedirs(directoryName, exist_ok=True)
      self.videoDirectory = directoryName
    else:
      # stop recording and record a new one
      self.stopRecordingVideo()

    if extra_filename is not None:
      output_video_filename = self.videoDirectory + '/' + datetime.datetime.now().strftime("vid-%Y-%m-%d-%H-%M-%S-%f") +extra_filename+ ".MP4"
    else:
      output_video_filename = self.videoDirectory + '/' + datetime.datetime.now().strftime("vid-%Y-%m-%d-%H-%M-%S-%f") + ".MP4"
    logID = self.startRecordingVideo(output_video_filename)
    self.videoLogID = logID


  def configure(self, args):
    self._args = args

  def seed(self, seed=None):
    self.np_random, seed = seeding.np_random(seed)
    return [seed]

  def _render_step_helper(self):
    """ Helper to configure the visualizer camera during step(). """
    # Sleep, otherwise the computation takes less time than real time,
    # which will make the visualization like a fast-forward video.
    time_spent = time.time() - self._last_frame_time
    self._last_frame_time = time.time()
    # time_to_sleep = self._action_repeat * self._time_step - time_spent
    time_to_sleep = self._time_step - time_spent
    #if time_to_sleep > 0 and (time_to_sleep < self._time_step):
      #time.sleep(time_to_sleep)
      
    base_pos = self.robot.GetBasePosition()
    camInfo = self._pybullet_client.getDebugVisualizerCamera()
    curTargetPos = camInfo[11]
    distance = camInfo[10]
    yaw = camInfo[8]
    pitch = camInfo[9]
    targetPos = [
        0.95 * curTargetPos[0] + 0.05 * base_pos[0], 0.95 * curTargetPos[1] + 0.05 * base_pos[1],
        curTargetPos[2]
    ]
    self._pybullet_client.resetDebugVisualizerCamera(distance, yaw, pitch, base_pos)

  def _configure_visualizer(self):
    """ Remove all visualizer borders, and zoom in """
    # default rendering options
    self._render_width = 960
    self._render_height = 720
    self._cam_dist = 1.0 
    self._cam_yaw = 0
    self._cam_pitch = -30 
    # get rid of visualizer things
    self._pybullet_client.configureDebugVisualizer(self._pybullet_client.COV_ENABLE_RGB_BUFFER_PREVIEW,0)
    self._pybullet_client.configureDebugVisualizer(self._pybullet_client.COV_ENABLE_DEPTH_BUFFER_PREVIEW,0)
    self._pybullet_client.configureDebugVisualizer(self._pybullet_client.COV_ENABLE_SEGMENTATION_MARK_PREVIEW,0)
    self._pybullet_client.configureDebugVisualizer(self._pybullet_client.COV_ENABLE_GUI,0)

  def render(self, mode="rgb_array", close=False):
    if mode != "rgb_array":
      return np.array([])
    base_pos = self.robot.GetBasePosition()
    view_matrix = self._pybullet_client.computeViewMatrixFromYawPitchRoll(
        cameraTargetPosition=base_pos,
        distance=self._cam_dist,
        yaw=self._cam_yaw,
        pitch=self._cam_pitch,
        roll=0,
        upAxisIndex=2)
    proj_matrix = self._pybullet_client.computeProjectionMatrixFOV(fov=60,
                                                                   aspect=float(self._render_width) /
                                                                   self._render_height,
                                                                   nearVal=0.1,
                                                                   farVal=100.0)
    (_, _, px, _,
     _) = self._pybullet_client.getCameraImage(width=self._render_width,
                                               height=self._render_height,
                                               viewMatrix=view_matrix,
                                               projectionMatrix=proj_matrix,
                                               renderer=pybullet.ER_BULLET_HARDWARE_OPENGL)
    rgb_array = np.array(px)
    rgb_array = rgb_array[:, :, :3]
    return rgb_array

  def addLine(self,lineFromXYZ,lineToXYZ,lifeTime=0,color=[1,0,0]):
    """ Add line between point A and B for duration lifeTime"""
    self._pybullet_client.addUserDebugLine(lineFromXYZ,
                                            lineToXYZ,
                                            lineColorRGB=color,
                                            lifeTime=lifeTime)

  def get_sim_time(self):
    """ Get current simulation time. """
    return self._sim_step_counter * self._time_step

  def scale_rand(self,num_rand,low,high):
    """ scale number of rand numbers between low and high """
    return low + np.random.random(num_rand) * (high - low)

  def add_random_boxes(self, num_rand=100, z_height=0.04):
    """Add random boxes in front of the robot in x [0.5, 20] and y [-3,3] """
    # x location
    x_low, x_upp = 0.5, 20
    # y location
    y_low, y_upp = -3, 3
    # block dimensions
    block_x_min, block_x_max = 0.1, 1
    block_y_min, block_y_max = 0.1, 1
    z_low, z_upp = 0.005, z_height
    # block orientations
    roll_low, roll_upp = -0.01, 0.01
    pitch_low, pitch_upp = -0.01, 0.01 
    yaw_low, yaw_upp = -np.pi, np.pi

    x = x_low + np.random.random(num_rand) * (x_upp - x_low)
    y = y_low + np.random.random(num_rand) * (y_upp - y_low)
    z = z_low + np.random.random(num_rand) * (z_upp - z_low)
    block_x = self.scale_rand(num_rand,block_x_min,block_x_max)
    block_y = self.scale_rand(num_rand,block_y_min,block_y_max)
    roll = self.scale_rand(num_rand,roll_low,roll_upp)
    pitch = self.scale_rand(num_rand,pitch_low,pitch_upp)
    yaw = self.scale_rand(num_rand,yaw_low,yaw_upp)
    # loop through
    for i in range(num_rand):
      sh_colBox = self._pybullet_client.createCollisionShape(self._pybullet_client.GEOM_BOX,
          halfExtents=[block_x[i]/2,block_y[i]/2,z[i]/2])
      orn = self._pybullet_client.getQuaternionFromEuler([roll[i],pitch[i],yaw[i]])
      block2=self._pybullet_client.createMultiBody(baseMass=0,baseCollisionShapeIndex = sh_colBox,
                            basePosition = [x[i],y[i],z[i]/2],baseOrientation=orn)
      # set friction coeff
      self._pybullet_client.changeDynamics(block2, -1, lateralFriction=self._ground_mu_k)

    # add walls 
    orn = self._pybullet_client.getQuaternionFromEuler([0,0,0])
    sh_colBox = self._pybullet_client.createCollisionShape(self._pybullet_client.GEOM_BOX,
        halfExtents=[x_upp/2,0.5,0.5])
    block2=self._pybullet_client.createMultiBody(baseMass=0,baseCollisionShapeIndex = sh_colBox,
                          basePosition = [x_upp/2,y_low,0.5],baseOrientation=orn)
    block2=self._pybullet_client.createMultiBody(baseMass=0,baseCollisionShapeIndex = sh_colBox,
                          basePosition = [x_upp/2,-y_low,0.5],baseOrientation=orn)

  def add_competition_blocks(self, num_stairs=100, stair_height=0.12, stair_width=0.25):
    """Wide, long so can't get around """
    y = 6
    block_x = stair_width * np.ones(num_stairs)
    block_z = np.arange(0,num_stairs)
    t = np.linspace(0,2*np.pi,num_stairs)
    block_z = stair_height*block_z/num_stairs * np.cos(block_z*np.pi/3 * t)
    curr_x = 1
    curr_z = 0 
    # loop through
    for i in range(num_stairs):
      curr_z = block_z[i]
      if curr_z > 0.005:
        sh_colBox = self._pybullet_client.createCollisionShape(self._pybullet_client.GEOM_BOX,
            halfExtents=[block_x[i]/2,y/2,curr_z/2])
        orn = self._pybullet_client.getQuaternionFromEuler([0,0,0])
        block2=self._pybullet_client.createMultiBody(baseMass=0,baseCollisionShapeIndex = sh_colBox,
                              basePosition = [curr_x,0,curr_z/2],baseOrientation=orn)
        # set friction coefficient 
        self._pybullet_client.changeDynamics(block2, -1, lateralFriction=self._ground_mu_k)

      curr_x += block_x[i]


  def _add_base_mass_offset(self, spec_mass=None, spec_location=None):
    """Attach mass to robot base."""
    quad_base = np.array(self.robot.GetBasePosition())
    quad_ID = self.robot.quadruped

    offset_low = np.array([-0.15, -0.05, -0.05])
    offset_upp = np.array([ 0.15,  0.05,  0.05])
    if spec_location is None:
      block_pos_delta_base_frame = self.scale_rand(3,offset_low,offset_upp)
    else:
      block_pos_delta_base_frame = np.array(spec_location)
    if spec_mass is None:
      base_mass = 8*np.random.random()
    else:
      base_mass = spec_mass
    if self._is_render:
      print('=========================== Random Mass:')
      print('Mass:', base_mass, 'location:', block_pos_delta_base_frame)
      # if rendering, also want to set the halfExtents accordingly 
      # 1 kg water is 0.001 cubic meters 
      boxSizeHalf = [(base_mass*0.001)**(1/3) / 2]*3
      translationalOffset = [0,0,0.1]
    else:
      boxSizeHalf = [0.05]*3
      translationalOffset = [0]*3

    sh_colBox = self._pybullet_client.createCollisionShape(self._pybullet_client.GEOM_BOX, 
                      halfExtents=boxSizeHalf, collisionFramePosition=translationalOffset)
    base_block_ID=self._pybullet_client.createMultiBody(baseMass=base_mass,
                                    baseCollisionShapeIndex = sh_colBox,
                                    basePosition = quad_base + block_pos_delta_base_frame,
                                    baseOrientation=[0,0,0,1])

    cid = self._pybullet_client.createConstraint(quad_ID, -1, base_block_ID, -1, 
          self._pybullet_client.JOINT_FIXED, [0, 0, 0], [0, 0, 0], -block_pos_delta_base_frame)
    # disable self collision between box and each link
    for i in range(-1,self._pybullet_client.getNumJoints(quad_ID)):
      self._pybullet_client.setCollisionFilterPair(quad_ID,base_block_ID, i,-1, 0)


def test_env():
  env = QuadrupedGymEnv(render=True, 
                        on_rack=True,
                        motor_control_mode='PD',
                        action_repeat=100,
                        )

  obs = env.reset()
  print('obs len', len(obs))
  action_dim = env._action_dim
  action_low = -np.ones(action_dim)
  print('act len', action_dim)
  action = action_low.copy()
  while True:
    action = 2*np.random.rand(action_dim)-1
    obs, reward, done, info = env.step(action)


if __name__ == "__main__":
  # test out some functionalities
  test_env()
  sys.exit()
